In [1]:
import sys
import glob
import os
import pandas as pd
import ee
import numpy as np
import geopandas as gp

import matplotlib.pyplot as plt
import rsfuncs as rs

from scipy import stats
from datetime import datetime, timedelta
from datetime import datetime as dt

%load_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = [18,16]
ee.Initialize()

In [2]:
# EE functions are in the rsfuncs module. Loacal functions are here: 

In [3]:
def read_sm_file(filename):
    with open(filename) as f:
        contents = f.readlines()

    data = []
    for line in contents:
        if line[0:1] == "#":
            continue
        else:
            data.append(line)

    headers = [x.replace("Soil Moisture Percent","smp").replace(" ","_") for x in data[0].split(",")]
    cols = [x.strip("\n").split(",") for x in data[1:]]

    df = pd.DataFrame(cols, columns = headers)
    
    return df

def get_ims_by_date(ims_list, var, res=10):
    imlist = []
    imdates = []
    num_images = len(ims_list.getInfo())

    for i in range (0, num_images):
        
        if i % 5 == 0:
            print(str((i / num_images)*100)[:5] + " % ")

        latlon = ee.Image.pixelLonLat().addBands(ims_list.get(i))
        imlist.append(array_from_latlon(latlon, var, res))
        date =  latlon.get('system:time_start')
        info_dict = ims.get(i).getInfo()
        date = info_dict['id']
        imdates.append(date)

    return imlist, imdates

def array_from_latlon(latlon_obj, var, res ):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,maxPixels=1e8,scale=res)
    try:
        lats = np.array((ee.Array(res.get("latitude")).getInfo()))
        lons = np.array((ee.Array(res.get("longitude")).getInfo()))
        data = np.array((ee.Array(res.get(var)).getInfo()))
    except:
        data = np.full_like(lats, np.nan,dtype=np.float64)
    
    out = make_np_array(data, lats, lons)
    return out   

def make_np_array(data, lats, lons):
    # get data from df as arrays
    lons = np.array(lons)
    lats = np.array(lats)
    data = np.array(data) # Set var here 
                                              
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)

    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)

    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]

    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32)

    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    
    return arr

def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-1, "day")
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj, area):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 


In [4]:
site_file = gp.read_file("../shape/scan_sites.shp")
sites = site_file[~site_file['state'].isin(["AK", "HI", "PR"])]

In [5]:
data = rs.load_data()

In [6]:
data_dir = "../data"

In [44]:
out_dict = {}

In [121]:
# For each site id, find the sm file with the data (in data dir), 
# calculate psi as psi = A * SM ^b (krishna's paper )
# query the (1) landcover, (2) Sentinel backscatter (Prism P), MODIS / Landsat LAI for the whole timeseries 
   
for idx, row in sites.iterrows():
    
    print("Processings site no {}".format(row.id))
    
    # Make geom to submit to EE 
    x,y = row.geometry.buffer(0.0001).envelope.exterior.coords.xy
    coords = [list(zip(x,y))]
    area = ee.Geometry.Polygon(coords)
    
    # Get the corresponding SCAN data file from data folder
    site_id = row.id
    sm_file = [os.path.join(data_dir,x) for x in os.listdir(data_dir) if site_id in x][0]
    sm_dat = read_sm_file(sm_file)
    sm_dat['Date'] =  pd.to_datetime(sm_dat['Date'], format='%Y%m%d %H:%M')
    sm_dat.set_index('Date', inplace=True)
        
    # start and end date
    startdate = sm_dat.index[0]
    enddate = sm_dat.index[-1]
    
    date = startdate.strftime("%Y-%m-%d")
        
    # Select the nlcd dataset
    dataset =  rs.load_data()['nlcd']
    ic = dataset[0]
    var = dataset[1]
    res = dataset[3]
    
    # find the nearest nlcd dataset 
    col = ic.filterDate(ee.Date(date).advance(-1, 'years'), ee.Date(date).advance(2, 'years')).first()
    if not col.getInfo():
        col = ic.filterDate(ee.Date(date).advance(-3, 'years'), ee.Date(date).advance(2, 'years')).first()
    t = col.reduceRegion(ee.Reducer.frequencyHistogram(), area, res).get(var)
    meta = col.getInfo()
    lc_class = int(list(t.getInfo().keys())[0])
    
    # Get the landcover type
    lcidx = meta['properties']['landcover_class_values'].index(lc_class)
    lctype = meta['properties']['landcover_class_names'][lcidx]
    print(lctype)
    lctypes.append(lctype)
    
    # Get Sentinel images and dates (ascending orbits only, VV polarization)
    s1 = rs.load_data()['s1']
    s1ic, s1var, s1res = s1[0], s1[1], s1[3]
    
    s1_col = s1ic.filterBounds(area).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    s1_col = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).select(s1var)
    
    # Krishna used ascending pass... I think descending is the correct orbit for the AM 
    
    s1_col = s1_col.filterDate(startdate,enddate)
    t = s1_col.sort('system:time_start')
    
    try:
        ims = t.toList(t.size())
        print("Processing {} sentinel overapasses".format(len(ims.getInfo())))
        s1dat, dates = get_ims_by_date(ims,s1var)
        
    except:
        print("no valid overpasses")
        continue
    
    # Calc the S1 backscatter in each image
    mean_sigmas = []
    std_sigmas = []

    for i in s1dat:
        mean_sigmas.append(np.mean(i))
        std_sigmas.append(np.std(i))
        
    # Convert the datestrings from S1 to pandas datetimes 
    for idx, x in enumerate(dates):
        timestamp = x.find("V_")+2
        timestr = x[timestamp:timestamp+13]
        dates[idx] = pd.to_datetime(timestr, format='%Y%m%d %H:%M')
        
    # Get PRISM data for all the S1 overpass dates to filter the rainy days
    print("processing PRISM")
    rainfall = []

    for i,x in enumerate(dates):
        if i % 5 == 0:
            print(str((i / len(dates))*100)[:5] + " % ")
        
        y,m,d = dates[i].year, dates[i].month, dates[i].day
        t = filter_date(rs.load_data()['prism_daily'][0], y, m, d).sum()
        precip_total = get_2day_precip(t, area)
        rainfall.append(precip_total)

    
    # MODIS LAI 
    print("processing MODIS")
    modis = rs.load_data()['modis_lai']
    mic, mvar, mres = modis[0],modis[1], modis[2]

    modis_lai = []
    for i,x in enumerate(dates):
        if i % 5 == 0:
            print(str((i / len(dates))*100)[:5] + " % ")
            
        y,m,d = dates[i].year, dates[i].month, dates[i].day

        start = ee.Date.fromYMD(y,m,d).advance(-2,"day")
        end = ee.Date.fromYMD(y,m,d).advance(2, "day")

        prod = mic.filterDate(start, end).sort('system:time_start', False).select(mvar).first()

        try:
            res = prod.reduceRegion(reducer=ee.Reducer.first(),geometry=area,scale=5) # Need to have scale smaller than geom
            data = np.array((ee.Array(res.get(mvar)).getInfo()))
            out = np.array(data)* 0.1
            modis_lai.append(out)
        except:
            out = np.nan
            modis_lai.append(np.nan)
            
    # Landsat - Note: some sites are in the overlap areas between passes.
    # these sites can have multiple obs / day or obs separated by 8days instead of 16. 
    
    print("Processing Landsat")
    landsat = rs.load_data()['l8_sr']
    lic, lvar, lsf = landsat[0],landsat[1], landsat[2]

    lstart = ee.Date.fromYMD(startdate.year,startdate.month,startdate.day).advance(-16,"day")
    lend = ee.Date.fromYMD(enddate.year,enddate.month,enddate.day).advance(16, "day")

    l8_col = lic.filterDate(lstart,lend).filterBounds(area).map(rs.mask_quality) # Mask clouds and shadows 
    lt = l8_col.sort('system:time_start')
    lims = lt.toList(lt.size())

    num_ims = len(lims.getInfo())

    ldfs = []

    for i in range (0, num_ims):
        if i % 5 == 0:
            print(str((i / num_ims)*100)[:5] + " % ")

        ls_latlon = ee.Image.pixelLonLat().addBands(lims.get(i))
        ltemp = ls_latlon.select(["B1", "B4", "B5"]).multiply(lsf)
        l8_res = ltemp.reduceRegion(reducer=ee.Reducer.mean(),geometry=area,bestEffort=True,scale=30)

        l8_info_dict = lims.get(i).getInfo()
        l8_date = l8_info_dict['id'][-8:]

        l8_out = l8_res.getInfo()

        ldf = pd.DataFrame.from_dict(l8_out.values()).T
        ldf.columns = l8_out.keys()
        ldf.index = pd.to_datetime([l8_date])
        ldfs.append(ldf)
        
    ls_dfs.append(ldfs)
                        
    # Filter the SCAN data for the S1 dates, 3 am - 7 am 
    rzsm = []
    ssm = []
    
    for i in dates:
        starttime = i.replace(second=0, microsecond=0, minute=0, hour=3)
        endtime = starttime+timedelta(hours= 4)
        df = pd.DataFrame(sm_dat[starttime:endtime])
        
        rzsm.append(df[df.columns[-1]].values)
        ssm.append(df[df.columns[-3]].values)
    
    # In case there are nans or data gaps in the sm data
    rzsm = [list(filter(None, x)) for x in rzsm]
    ssm = [list(filter(None, x)) for x in ssm]
    
    # Calc the 5 hour mean for each sensor reading 
    for idx,x in enumerate(rzsm):
        rzsm[idx] = np.nanmean([np.float(i) for i in x])
    
    for idx,x in enumerate(ssm):
        ssm[idx] = np.nanmean([np.float(i) for i in x])

    # params to calculate psi 
    a = row.a
    b = row.b
    
    # RZ and Surface LWP = A * sm ^b
    psi1 = [a*(x/100)**b for x in rzsm]
    psi2 = [a*(x/100)**b for x in ssm]
    
    # Make a dataframe out of everything
    df = pd.DataFrame([dates, mean_sigmas, std_sigmas, rainfall, modis_lai, rzsm,psi1,ssm,psi2])
    df = df.T
    df.columns = (['date', "sigma", "std_sigma", "precip","LAI", "rzsm","psi_rz", "ssm", "psi_s"])
    df = rs.col_to_dt(df) # set the date col as datetime index 
    mask=(df['precip'] < 0.1)
    dfout = df[mask]

    # For now append the nearest landsat image band values to the master df 
    # TODO: Golay interpolation 
    
    ls = pd.concat(ldfs)
    ls_df = ls[(ls.T != 0).any()]
    ls_df = ls_df.groupby(level = 0).mean()
    
    # Break the loop if there are no non- rainy sentinel overpasses 
    if dfout.empty:
        print("No non-rainy overpaasses ")
        continue 
        
    l8_dfs = []
    
    for i in dfout.index:
        l8_idx = ls_df.index.get_loc(i, method='nearest')
        l8_contemp = ls_df.iloc[l8_idx]
        t = pd.DataFrame(l8_contemp).T
        l8_dfs.append(t)
    
    fin_ls = pd.concat(l8_dfs)

    FIN = pd.concat( [dfout.reset_index(drop=True), fin_ls.reset_index(drop=True)], axis=1) 
    FIN.index = dfout.index

    print(FIN.head())
    
    out_dict[(row.id)] = FIN
    

Processings site no 2057
Deciduous Forest - Areas dominated by trees generally greater than 5 meters tall, and greater than 20% of total vegetation cover. More than 75 percent of the tree species shed foliage simultaneously in response to seasonal change.
Processing 2 sentinel overapasses
0.0 % 
processing PRISM
0.0 % 
processing MODIS
0.0 % 
Processing Landsat
0.0 % 
3.401 % 
6.802 % 
10.20 % 
13.60 % 
17.00 % 
20.40 % 
23.80 % 
27.21 % 
30.61 % 
34.01 % 
37.41 % 
40.81 % 
44.21 % 
47.61 % 
51.02 % 
54.42 % 
57.82 % 
61.22 % 
64.62 % 
68.02 % 
71.42 % 
74.82 % 
78.23 % 
81.63 % 
85.03 % 
88.43 % 
91.83 % 
95.23 % 
98.63 % 
                       sigma std_sigma precip  LAI   rzsm    psi_rz    ssm  \
date                                                                         
2016-09-02 11:48:00 -6.42287   3.70825    0.0  5.8  44.24  0.198571  38.04   

                        psi_s        B1        B4        B5  
date                                                         
2016-09-0

/anaconda3/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:181: RuntimeWarning: Mean of empty slice


No non-rainy overpaasses 
Processings site no 2174
Sedge/Herbaceous - Alaska only areas dominated by sedges and forbs, generally greater than 80% of total vegetation. This type can occur with significant other grasses or other grass like plants, and includes sedge tundra, and sedge tussock tundra.
Processing 6 sentinel overapasses
0.0 % 
83.33 % 
processing PRISM
0.0 % 
83.33 % 
processing MODIS
0.0 % 
83.33 % 
Processing Landsat
0.0 % 
6.666 % 
13.33 % 
20.0 % 
26.66 % 
33.33 % 
40.0 % 
46.66 % 
53.33 % 
60.0 % 
66.66 % 
73.33 % 
80.0 % 
86.66 % 
93.33 % 
                       sigma std_sigma precip LAI  rzsm    psi_rz    ssm  \
date                                                                       
2016-09-02 11:48:00 -6.34905   2.84428    0.0   2  47.5  0.766355  37.68   

                       psi_s        B1        B4      B5  
date                                                      
2016-09-02 11:48:00  13.2991  0.020196  0.041975  0.4072  
Processings site no 2055
Evergr

/anaconda3/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:178: RuntimeWarning: Mean of empty slice


                       sigma std_sigma precip  LAI   rzsm      psi_rz    ssm  \
date                                                                           
2016-09-02 11:49:00 -8.71414   5.07461    0.0    1  30.76  0.00478776   18.7   
2017-09-15 11:48:00 -8.16496   4.72867    0.0  1.2    NaN         NaN  21.92   
2018-10-16 11:48:00 -8.05903   4.82977    0.0  1.2  31.58  0.00419454  22.08   

                         psi_s        B1        B4        B5  
date                                                          
2016-09-02 11:49:00  0.0584639  0.061100  0.090600  0.472657  
2017-09-15 11:48:00  0.0263007  0.020073  0.044313  0.502877  
2018-10-16 11:48:00  0.0253563  0.036383  0.073883  0.303983  
Processings site no 2182
Grassland/Herbaceous - Areas dominated by grammanoid or herbaceous vegetation, generally greater than 80% of total vegetation.  These areas are not subject to intensive management such as tilling, but can be utilized for grazing.
Processing 6 sentinel overapa

/anaconda3/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:189: RuntimeWarning: divide by zero encountered in double_scalars


                       sigma std_sigma precip  LAI  rzsm   psi_rz   ssm  \
date                                                                      
2015-01-19 13:58:00 -10.0718   4.76805    0.0  0.1   8.9  24.5333     0   
2015-02-12 13:58:00 -10.5645   4.74604    0.0  0.1   8.5  28.3443  9.66   
2015-03-08 13:58:00 -10.0899    4.7388    0.0  0.2  8.82  25.2389  8.24   
2015-04-01 13:58:00 -10.9277   5.06034    0.0  0.2  9.12  22.7226  7.54   
2015-04-25 13:58:00 -13.8486   6.23544    0.0  0.2  9.42  20.5268   5.8   

                       psi_s      B1      B4      B5  
date                                                  
2015-01-19 13:58:00      inf  0.0367  0.0781  0.1790  
2015-02-12 13:58:00  18.9676  0.0413  0.0907  0.1808  
2015-03-08 13:58:00  31.2486  0.0391  0.0990  0.1832  
2015-04-01 13:58:00  41.2948  0.3822  0.4012  0.4778  
2015-04-25 13:58:00  94.1191  0.0499  0.1149  0.2067  
Processings site no 2192
Dwarf Scrub - Alaska only areas dominated by shrubs less than 20

/anaconda3/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:188: RuntimeWarning: divide by zero encountered in double_scalars


                       sigma std_sigma precip  LAI   rzsm    psi_rz    ssm  \
date                                                                         
2016-01-16 12:05:00 -5.43631    3.1565    0.0  0.4  25.58  0.894235  28.44   
2016-01-28 12:05:00 -11.6233   6.80644    0.0  0.4  19.06   3.33352  32.14   
2016-01-28 12:05:00 -11.6214   6.77002    0.0  0.4  19.06   3.33352  32.14   
2016-02-09 12:05:00 -7.46733    4.3944    0.0  0.7     20   2.68783  28.92   
2017-05-04 12:04:00 -6.16819    3.5618    0.0  3.2  32.16  0.321249  16.58   

                        psi_s        B1        B4        B5  
date                                                         
2016-01-16 12:05:00  0.556671  0.012877  0.043439  0.148624  
2016-01-28 12:05:00  0.322144  0.026032  0.058350  0.197018  
2016-01-28 12:05:00  0.322144  0.026032  0.058350  0.197018  
2016-02-09 12:05:00  0.516525  0.026032  0.058350  0.197018  
2017-05-04 12:04:00   6.21792  0.034179  0.045382  0.342727  
Processings site no

IndexError: index 0 is out of bounds for axis 0 with size 0

In [156]:
# How many rows of data?

nrows = 0

for k,v in out_dict.items():
    nrows+=len(v)
    
print(str(nrows) + " rows")

# How many sites have data, how many dont? 
files1 = [x for x in os.listdir("../data")]
success = [k+".txt" for k,v in out_dict.items()]
nodat = [x for x in files1 if x not in success]

print(str(len(nodat)) + " sites have no data")

print(str(len(success)) + " sites have data")

3617 rows
143 sites have no data
86 sites have data


In [131]:
# Save 

In [137]:
for k,v in out_dict.items():
    v.to_csv("train/{}".format(str(k)+ ".txt"))

In [ ]:
df2 = ls[(ls.T != 0).any()]

In [ ]:
ls = pd.concat(ls_dfs[1])
df2 = ls[(ls.T != 0).any()]

df = rs.col_to_dt(df)
ls_df = ls[(ls.T != 0).any()]

for i in df.index:
    l8_idx = df2.index.get_loc(i, method='nearest')
    l8_contemp = df2.iloc[l8_idx]
    t = pd.DataFrame(l8_contemp).T
    t.reset_index(drop=True, inplace=True)
    df[t.columns] = t.values

In [ ]:
df

In [ ]:
df[t.columns] = t.values

In [ ]:
df

In [ ]:
startdate = sm_dat.index[0]
enddate = sm_dat.index[-1]
    

landsat = rs.load_data()['l8_sr']
lic, lvar, lsf = landsat[0],landsat[1], landsat[2]

lstart = ee.Date.fromYMD(startdate.year,startdate.month,startdate.day).advance(-16,"day")
lend = ee.Date.fromYMD(enddate.year,enddate.month,enddate.day).advance(16, "day")

l8_col = lic.filterDate(lstart,lend).filterBounds(area).map(rs.mask_quality)
lt = l8_col.sort('system:time_start')
lims = lt.toList(lt.size())

num_ims = len(lims.getInfo())

ldfs = []

for i in range (0, num_ims):
    if i % 5 == 0:
        print(str((i / num_ims)*100)[:5] + " % ")

    ls_latlon = ee.Image.pixelLonLat().addBands(lims.get(i))
    ltemp = ls_latlon.select(["B1", "B2", "B3", "B4", "B5", "B6", "B7"]).multiply(lsf)
    l8_res = ltemp.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,maxPixels=1e8,scale=30)

    l8_info_dict = lims.get(i).getInfo()
    l8_date = l8_info_dict['id'][-8:]

    l8_out = l8_res.getInfo()

    ldf = pd.DataFrame.from_dict(l8_out.values()).T
    ldf.columns = l8_out.keys()
    ldf.index = pd.to_datetime([l8_date])
    ldfs.append(ldf)
    print(ldf)

In [ ]:
lims.get(2).getInfo()['properties']['system:index']

In [ ]:
lims.get(3).getInfo()['properties']['system:index']

In [ ]:
lims.get(10).getInfo()['id']

In [ ]:
lims.get(11).getInfo()['id']

In [ ]:
lims.get(0).getInfo()['properties']['system:index']

In [ ]:
data = rs.load_data()
landsat = data['l8_sr']
lic, lvar, lsf = landsat[0],landsat[1], landsat[2]

start = ee.Date.fromYMD(startdate.year,startdate.month,startdate.day).advance(8,"day")
end = ee.Date.fromYMD(enddate.year,enddate.month,enddate.day).advance(8, "day")

l8_col = lic.filterDate(start,end).filterBounds(area).map(rs.mask_quality)
t = l8_col.sort('system:time_start')
ims = t.toList(t.size())

num_ims = len(ims.getInfo())

ldfs = []

for i in range (0, num_ims):
    if i % 5 == 0:
        print(str((i / num_ims)*100)[:5] + " % ")

    latlon = ee.Image.pixelLonLat().addBands(ims.get(i))
    temp = latlon.select(["B1", "B2", "B3", "B4", "B5", "B6", "B7"]).multiply(lsf)
    res = temp.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,scale=30)
    info_dict = lims.get(i).getInfo()
    l8_date = info_dict['id'][-8:]

    l8_out = res.getInfo()

    ldf = pd.DataFrame.from_dict(l8_out.values()).T
    ldf.columns = l8_out.keys()
    ldf.index = pd.to_datetime([l8_date])
    ldfs.append(ldf)
    
    print(ldf.head())

In [ ]:
l8_out = res.getInfo()

ldf = pd.DataFrame.from_dict(l8_out.values()).T
ldf.columns = l8_out.keys()
ldf.index = pd.to_datetime([l8_date])

In [ ]:
l8_d1.getInfo()

In [ ]:
data = rs.load_data()

landsat = data['l8_sr']
lic, lvar, lres = landsat[0],landsat[1], landsat[2]

sd = 

l8_col = lic.filterDate(ee.Date(startdate).advance("day", -16),ee.Date(enddate).advance("day", 16)).filterBounds(area)
t = l8_col.sort('system:time_start')
ims = t.toList(t.size())

num_ims = len(ims.getInfo())

In [ ]:
imlist = []

for i in range (0, num_ims):
        if i % 5 == 0:
            print(str((i / num_ims)*100)[:5] + " % ")

        latlon = ee.Image.pixelLonLat().addBands(ims.get(i))
        temp = latlon.select(["B1", "B2", "B3", "B4", "B5", "B6", "B7"]).getInfo()
        res = temp.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,maxPixels=1e8,scale=30)

        
#         imlist.append(array_from_latlon(latlon, "B7", 30))
#         date =  latlon.get('system:time_start')
#         info_dict = ims.get(i).getInfo()
#         date = info_dict['id']
#         imdates.append(date)


In [ ]:
temp = latlon.select(["B1", "B2", "B3", "B4", "B5", "B6", "B7"])

In [ ]:
res = temp.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,maxPixels=1e8,scale=30)


In [ ]:
res.getInfo()

In [ ]:
    imlist = []
    imdates = []
    num_images = len(ims_list.getInfo())

    for i in range (0, num_images):
        
        if i % 5 == 0:
            print(str((i / num_images)*100)[:5] + " % ")

        latlon = ee.Image.pixelLonLat().addBands(ims_list.get(i))
        imlist.append(array_from_latlon(latlon, var, res))
        date =  latlon.get('system:time_start')
        info_dict = ims.get(i).getInfo()
        date = info_dict['id']
        imdates.append(date)

    return imlist, imdates

In [ ]:
landsat = data['l8_sr']
lic, lvar, lres = landsat[0],landsat[1], landsat[2]

for i,x in enumerate(dates[:1]):
    if i % 5 == 0:
        print(str((i / len(dates))*100)[:5] + " % ")

    y,m,d = dates[i].year, dates[i].month, dates[i].day
    
    start = ee.Date.fromYMD(y,m,d).advance(-8,"day")
    end = ee.Date.fromYMD(y,m,d).advance(8, "day")

    prod = lic.filterDate(start, end).sort('system:time_start', False).select(mvar).first()
    l8_im= prod.filterBounds(area).map(rs.mask_quality)


In [ ]:
    s1_col = s1_col.filterDate(startdate,enddate)
    t = s1_col.sort('system:time_start')
    
    try:
        ims = t.toList(t.size())

In [ ]:
    band_name = col.select(band).median()
    latlon = ee.Image.pixelLonLat().addBands(band_name).multiply(0.0001)

    # apply reducer to list
    latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=bounds,
      maxPixels=1e13,
      scale=res)
    
    data = np.array((ee.Array(latlon.get(band)).getInfo()))
    lats = np.array((ee.Array(latlon.get("latitude")).getInfo()))
    lons = np.array((ee.Array(latlon.get("longitude")).getInfo()))
    
    arr = array_from_coords(data,lats,lons)

In [ ]:



for i,x in enumerate(dates):
    if i % 5 == 0:
        print(str((i / len(dates))*100)[:5] + " % ")

    y,m,d = dates[i].year, dates[i].month, dates[i].day

    start = ee.Date.fromYMD(y,m,d).advance(-2,"day")
    end = ee.Date.fromYMD(y,m,d).advance(2, "day")

    prod = mic.filterDate(start, end).sort('system:time_start', False).select(mvar).first()

In [ ]:
x = np.array(mean_sigmas)
x = x.reshape(x.shape[0])
y = np.array(rzsm)
y = y.reshape(y.shape[0])
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

In [ ]:
plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture 
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], "10"))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(mean_sigmas, rzsm, label = 'rzsm')
plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()

In [ ]:
# as a timeseries

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''$\sigma$ vs Soil moisture Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:red'
ax1.set_xlabel('date')
ax1.set_ylabel("$\sigma$ (dB)", color=color)
ax1.plot(dates, mean_sigmas, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# remove the days with high preceding rainfall 

In [ ]:
imageCollection = ee.ImageCollection("OREGONSTATE/PRISM/AN81d").filterBounds(area)

In [ ]:
def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-1, "day")
    end = ee.Date.fromYMD(y,m,d)
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

In [ ]:
rainfall = []

for i,x in enumerate(dates):
    y,m,d = dates[i].year, dates[i].month, dates[i].day
    t = filter_date(imageCollection, y, m, d).sum()
    precip_total = get_2day_precip(t)
    rainfall.append(precip_total)

In [ ]:
fdf = pd.DataFrame([dates, mean_sigmas, rzsm, rainfall])

In [ ]:
fdf = fdf.T
fdf.columns = (['date', "sigma", "rzsm", "rainfall"])

In [ ]:
mask=(fdf['rainfall'] < 0.1)
asdf=fdf[mask]

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''2 day sum Precip (mm) and Soil moisture (%) Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:blue'
ax1.set_xlabel('date')
ax1.set_ylabel("Precip (mm)", color=color)
ax1.plot(dates, rainfall, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# Plot linear fit of sigma vs SM
x = np.array(asdf['sigma'])
x = x.reshape(x.shape[0]).astype(np.float32)
y = np.array(asdf['rzsm'])
y = y.reshape(y.shape[0]).astype(np.float32)

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture for overpasses without prior precipitation
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(asdf), asdf['date'].min(), asdf['date'].max(), buffer_size))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(asdf['sigma'],asdf['rzsm'], label = "rzsm")
# plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()

In [ ]:
# Let's test with 4 day precip accumulation

imageCollection = ee.ImageCollection("OREGONSTATE/PRISM/AN81d").filterBounds(area)

def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-4, "day")
    end = ee.Date.fromYMD(y,m,d)
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

rainfall = []

for i,x in enumerate(dates):
    y,m,d = dates[i].year, dates[i].month, dates[i].day
    t = filter_date(imageCollection, y, m, d).sum()
    precip_total = get_2day_precip(t)
    rainfall.append(precip_total)
    

In [ ]:
fdf = pd.DataFrame([dates, mean_sigmas, rzsm, rainfall])
fdf = fdf.T
fdf.columns = (['date', "sigma", "rzsm", "rainfall"])
mask=(fdf['rainfall'] < 0.1)
asdf=fdf[mask]

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''4 day sum Precip (mm) and Soil moisture (%) Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:blue'
ax1.set_xlabel('date')
ax1.set_ylabel("3 day precipitation (mm)", color=color)
ax1.plot(dates, rainfall, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# Plot linear fit of sigma vs SM
x = np.array(asdf['sigma'])
x = x.reshape(x.shape[0]).astype(np.float32)
y = np.array(asdf['rzsm'])
y = y.reshape(y.shape[0]).astype(np.float32)

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture for overpasses without prior precipitation (4 days)
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(asdf), asdf['date'].min(), asdf['date'].max(), buffer_size))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(asdf['sigma'],asdf['rzsm'], label = "rzsm")
# plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()